In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import json

%matplotlib inline

# Analysis of results

## Import of data

In [2]:
DATE = "2020-12-17"


def import_native_and_bytecode(filename):
    native = []
    bytecode = []
    with open(filename) as f:
        for line in f:
            entry = json.loads(line)
            name_sections = list(entry['name'].split('.'))
            if name_sections[0].endswith('bytecode'):
                name_sections[0] = name_sections[0][:-9]
                entry['name'] = '.'.join(name_sections)
                bytecode.append(entry)
            else:
                native.append(entry)
    return pd.json_normalize(native), pd.json_normalize(bytecode)


def import_just_bytecode(filename):
    data = []
    for line in open(filename):
        entry = json.loads(line)
        name_sections = list(entry['name'].split('.'))
        name_sections[0] = name_sections[0][:-9]
        entry['name'] = '.'.join(name_sections)
        data.append(entry)
    return pd.json_normalize(data)
            
    
native, stock = import_native_and_bytecode(f'{DATE}/4.11.1+stock.orun.bench')
compile_only = import_just_bytecode(f'{DATE}/4.11.1+jit+compileonly.orun.bench')
jit_disabled = import_just_bytecode(f'{DATE}/4.11.1+jit+disabled.orun.bench')
jit = import_just_bytecode(f'{DATE}/4.11.1+jit.orun.bench')

In [3]:
compile_only

,name,command,time_secs,user_time_secs,sys_time_secs,maxrss_kB,codesize,ocaml_url,ocaml.version,ocaml.c_compiler,...,gc.allocated_words,gc.minor_words,gc.promoted_words,gc.major_words,gc.minor_collections,gc.major_collections,gc.heap_words,gc.heap_chunks,gc.top_heap_words,gc.compactions
0,imrin_mem_rw.10_000_50_000_20_100_000_000,taskset --cpu-list 5 ./irmin_mem_rw.bc 10_000 ...,31.004070,30.963376,0.000000,25096,0.0,../../src,4.11.1+dev_jit,gcc,...,4761174129,4761149266,792477,817340,18166,8,661504,9,661504,0
1,kernel2.3_1,taskset --cpu-list 5 ./kernel2.bc 3 1,0.006635,0.006450,0.000000,6440,0.0,../../src,4.11.1+dev_jit,gcc,...,10144,7778,0,2366,0,0,126976,1,126976,0
2,revcomp2.,taskset --cpu-list 5 ./revcomp2.bc,8.242040,8.182008,0.049960,213236,0.0,../../src,4.11.1+dev_jit,gcc,...,79174867,79171295,49848350,49851922,310,18,25093632,35,25093632,0
3,test_decompress.64_524_288,taskset --cpu-list 5 ./test_decompress.bc 64 5...,35.245702,35.200695,0.003333,14228,0.0,../../src,4.11.1+dev_jit,gcc,...,1157354718,1051653823,154460,105855355,4299,800,661504,8,843776,62
4,kernel3.3_1_1,taskset --cpu-list 5 ./kernel3.bc 3 1 1,0.006911,0.003362,0.003343,6488,0.0,../../src,4.11.1+dev_jit,gcc,...,10926,8548,0,2378,0,0,126976,1,126976,0
5,imrin_mem_rw.10_000_50_000_80_100_000_000,taskset --cpu-list 5 ./irmin_mem_rw.bc 10_000 ...,35.105648,35.054328,0.009982,25140,0.0,../../src,4.11.1+dev_jit,gcc,...,6441133781,6441108918,907422,932285,24576,10,661504,9,661504,0
6,thread_ring_lwt_mvar.20_000,taskset --cpu-list 5 ./thread_ring_lwt_mvar.bc...,10.858525,10.842090,0.003332,9172,0.0,../../src,4.11.1+dev_jit,gcc,...,1146881274,1146875403,118963458,118969329,5251,1749,188416,2,188416,0
7,yojson_ydump.sample.json,taskset --cpu-list 5 ./ydump.bc -c sample.json,1.211749,1.199981,0.009992,14732,0.0,../../src,4.11.1+dev_jit,gcc,...,12167404,9937912,1804767,4034259,44,17,568832,9,752640,5
8,zarith_pi.5000,taskset --cpu-list 5 ./zarith_pi.bc 5000,0.645017,0.348238,0.295308,21188,0.0,../../src,4.11.1+dev_jit,gcc,...,396131833,3425517,20904,392727220,1075,640,1317376,15,1761792,212


In [4]:
jit.sort_values(by='time_secs', ascending=False)

,name,command,time_secs,user_time_secs,sys_time_secs,maxrss_kB,codesize,ocaml_url,ocaml.version,ocaml.c_compiler,...,gc.allocated_words,gc.minor_words,gc.promoted_words,gc.major_words,gc.minor_collections,gc.major_collections,gc.heap_words,gc.heap_chunks,gc.top_heap_words,gc.compactions
5,imrin_mem_rw.10_000_50_000_80_100_000_000,taskset --cpu-list 5 ./irmin_mem_rw.bc 10_000 ...,24.049739,24.011971,0.009972,25112,0.0,../../src,4.11.1+dev_jit,gcc,...,6441133781,6441108918,907422,932285,24576,10,661504,9,661504,0
0,imrin_mem_rw.10_000_50_000_20_100_000_000,taskset --cpu-list 5 ./irmin_mem_rw.bc 10_000 ...,19.300709,19.271400,0.006661,25144,0.0,../../src,4.11.1+dev_jit,gcc,...,4761174129,4761149266,792477,817340,18166,8,661504,9,661504,0
3,test_decompress.64_524_288,taskset --cpu-list 5 ./test_decompress.bc 64 5...,14.573250,14.547763,0.006663,14152,0.0,../../src,4.11.1+dev_jit,gcc,...,1157354718,1051653823,154460,105855355,4299,800,661504,8,843776,62
6,thread_ring_lwt_mvar.20_000,taskset --cpu-list 5 ./thread_ring_lwt_mvar.bc...,5.282927,5.273267,0.003318,9300,0.0,../../src,4.11.1+dev_jit,gcc,...,1146881274,1146875403,118963458,118969329,5251,1749,188416,2,188416,0
2,revcomp2.,taskset --cpu-list 5 ./revcomp2.bc,3.394972,3.340521,0.049928,213256,0.0,../../src,4.11.1+dev_jit,gcc,...,79174867,79171295,49848350,49851922,310,18,25093632,35,25093632,0
8,zarith_pi.5000,taskset --cpu-list 5 ./zarith_pi.bc 5000,0.638645,0.310246,0.326963,21032,0.0,../../src,4.11.1+dev_jit,gcc,...,396131833,3425517,20904,392727220,1075,640,1317376,15,1761792,212
7,yojson_ydump.sample.json,taskset --cpu-list 5 ./ydump.bc -c sample.json,0.560969,0.546685,0.013250,14796,0.0,../../src,4.11.1+dev_jit,gcc,...,12167404,9937912,1804767,4034259,44,17,568832,9,752640,5
4,kernel3.3_1_1,taskset --cpu-list 5 ./kernel3.bc 3 1 1,0.006693,0.003234,0.003255,6480,0.0,../../src,4.11.1+dev_jit,gcc,...,10926,8548,0,2378,0,0,126976,1,126976,0
1,kernel2.3_1,taskset --cpu-list 5 ./kernel2.bc 3 1,0.006608,0.003222,0.003202,6508,0.0,../../src,4.11.1+dev_jit,gcc,...,10144,7778,0,2366,0,0,126976,1,126976,0


In [5]:
def add_suffix(df, suffix):
    return df.add_suffix(suffix).rename(columns={f'name{suffix}': 'name'})

combined = (
    add_suffix(native, '_native')
    .merge(add_suffix(stock, '_stock'), on='name')
    .merge(add_suffix(jit_disabled, '_jit_disabled'), on='name')
    .merge(add_suffix(compile_only, '_compile_only'), on='name')
    .merge(add_suffix(jit, '_jit'), on='name')
)[['name', 'time_secs_native', 'time_secs_stock', 'time_secs_jit_disabled', 'time_secs_compile_only', 'time_secs_jit']]
# combined['speedup'] = combined.time_secs_bytecode / combined.time_secs_native
# combined = combined.sort_values(by='speedup', ascending=False).reset_index(drop=True)
print(combined.to_string())

                                        name  time_secs_native  time_secs_stock  time_secs_jit_disabled  time_secs_compile_only  time_secs_jit
0                                  revcomp2.          1.340486         5.182203                8.531131                8.242040       3.394972
1  imrin_mem_rw.10_000_50_000_80_100_000_000          3.440607        26.745500               37.075399               35.105648      24.049739
2                thread_ring_lwt_mvar.20_000          1.665354         7.995121               11.814604               10.858525       5.282927
3                             zarith_pi.5000          0.593269         0.616416                0.617576                0.645017       0.638645
4                                kernel2.3_1          0.001877         0.002670                0.002496                0.006635       0.006608
5                 test_decompress.64_524_288          1.629639        25.151667               37.322547               35.245702      14.573250

In [31]:
combined['speedup'].mean()

5.230522939611381